# Natural Language Inference using Neural Networks - VG
Based on the Python Notebook by Adam Ek, expanded upon by Maria Irena Szawerna for the VT2022 Computational Semantics course.

----------------------------------

# 1. Data

##### Problem description, references
In this notebook I will, on the basis of Lab 5, explore how SNLI-trained models perform on FRACAS and whether or not fine-tuning them will improve the performance. A more detailed summary can be found at the end of the notebook. Throughout the notebook I will remark on what was sourced from the original Lab 5 notebook or somewhere else; I did contribute to all the parts of the basic Lab 5 though, so it is not strictly only my classmates' work. The references for the lab can be found at the end of the notebook, and I did not use anything besides that except for the FRACAS dataset itself, which is linked later in this cell, as well as a guide on how to access XML files, linked in the appropriate code block.  

##### Research question and thesis statement  
+ Does an SNLI-trained BiLSTM model perform well when evaluated on the FRACAS dataset? Does fine-tuning it on FRACAS influence the performance?
+ My prediction is that fine-tuning a model trained on SNLI on FRACAS will improve its performance on FRACAS-related tasks. However, the FRACAS dataset is much smaller, so the change may not be drastically big.

##### Description of the datasets' structure and my changes to the FRACAS dataset
*The dataset could not be downloaded in the simplified version, the link did not work; instead, I used [this](https://github.com/sdobnik/computational-semantics/blob/master/assignments/05-natural-language-inference/simple_snli_1.0.zip)*

The (simplified) data is organized as follows (tab-separated values):
* Column 1: Premise
* Column 2: Hypothesis
* Column 3: Relation

I will also work with the FRACAS dataset from [here](https://nlp.stanford.edu/~wcmac/downloads/fracas.xml). This one accepts the following relations:
* yes - equivalent to entailment
* no - equivalent to contradiction
* unknown - equivalent to neutral (neither entailment nor contradiction can be concluded)
* undef - the example is too tricky for an expert to resolve  

I will have to transform this dataset into a format that will be easy to work with, replacing their names of judgements with mine. In addition, some of their examples have two or more premises for one hypothesis. According to the dataset decription, one-premise problems constitute around 55% of the dataset, that is 192 problems. While this does slim the dataset down, I cannot do it any other way without combining the multiple premises into one sentence, as the model I am working with only compares two sentences (the hypothesis and the premise), and so the multi-premise problems will have to be excluded. In the end fewer than 192 problems are used, as some of those do not have an answer (undef), and so I excluded them.  

----

In [1]:
# This is the same as in Lab 5
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 
import torch
import torch.optim as optim
import torch.nn as nn

device = torch.device('cuda:2')

batch_size = 8
learning_rate = 0.001
epochs = 3

import random
import math

import numpy as np
import pandas as pd

### SNLI data and datasets/loaders  
----
**THIS WHOLE SECTION IS THE SAME AS IN LAB 5**

In [2]:
train_data = pd.read_csv('simple_snli_1.0_train.csv', header=None, sep='\t')
train_data.columns = ['premise', 'hypothesis', 'relation']
test_data = pd.read_csv('simple_snli_1.0_test.csv', header=None, sep='\t')
test_data.columns = ['premise', 'hypothesis', 'relation']
dev_data = pd.read_csv('simple_snli_1.0_dev.csv', header=None, sep='\t')
dev_data.columns = ['premise', 'hypothesis', 'relation']

In [3]:
from torch.utils.data import DataLoader, Dataset

In [4]:
# I implement a Dataset to keep track of vocab, word2idx, idx2word
# Dataset can also be used in DataLoader which gives batch loading, etc, for free.

class InferenceDataset(Dataset):

    def __init__(self, data, unk_label='<unk>', pad_label='<pad>'):
        
        self.unk_idx, self.unk_label = 0, unk_label
        self.pad_idx, self.pad_label = 1, pad_label

        self.data = data.copy()
        self.data['premise'] = self.data['premise'].apply(self.tokenize)
        self.data['hypothesis'] = self.data['hypothesis'].apply(self.tokenize)


        self.vocab = self.__unique_words()
        
        self.word2idx = dict()
        self.idx2word = dict()
        self.word2idx[self.unk_label] = self.unk_idx
        self.word2idx[self.pad_label] = self.pad_idx
        self.word2idx.update({word:idx+max(self.word2idx.values())+1 for idx, word in enumerate(self.vocab)})

        self.idx2word = {v:k for k,v in self.word2idx.items()}

        self.labels = list(np.unique(self.data['relation']))

    def __unique_words(self):
        all_words = []
        for s in self.data['premise']:
            all_words += s
        for s in self.data['hypothesis']:
            all_words += s
        return np.unique(all_words)
        
    def tokenize(self, string):
        if isinstance(string, str): 
            # The tokenizer was given as a whitespace tokenizer
            return string.lower().split()
        else:  # for NaN
            return "<unk>"

    def __getitem__(self, idx):
        #x = self.data.iloc[0] #for test
        x = self.data.iloc[idx]
        out = (x['premise'], x['hypothesis'], x['relation'])
        return out
        
    def __len__(self):
        return len(self.data)

In [5]:
from collections import namedtuple
from torch.nn.utils.rnn import pad_sequence 

relation_to_idx = {k:v for v,k in enumerate(sorted(np.unique(train_data['relation'])))}
idx_relation = {v:k for k,v in relation_to_idx.items()}

class Collate():
    def __init__(self, word_to_idx, pad_idx=1, unk_idx=0, relation_to_idx=relation_to_idx):
        self.pad_idx = pad_idx
        self.unk_idx = unk_idx
        self.word_to_idx = word_to_idx
        self.relation_to_idx = relation_to_idx
    def __call__(self, batch):
        batch = np.transpose(batch)
        
        premises = np.transpose(batch[0])
        premises = [torch.tensor([self.word_to_idx.get(w, self.unk_idx) for w in s], device=device) for s in premises]
        premises = pad_sequence(premises, batch_first=True, padding_value=self.pad_idx)

        hypothesis = np.transpose(batch[1]) #batch first
        hypothesis = [torch.tensor([self.word_to_idx.get(w, self.unk_idx) for w in s], device=device) for s in hypothesis]
        hypothesis = pad_sequence(hypothesis, batch_first=True, padding_value=self.pad_idx)
        
        relations = [self.relation_to_idx[rel] for rel in batch[2]]

        return premises, hypothesis, relations


def dataloader(dataset, word2idx, pad_idx, unk_idx, batch_size=32, shuffle=True): # Need word2idx etc to match between train and test. Id probably do this is another wya in hindsight.
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        collate_fn=Collate(word2idx, pad_idx, unk_idx) )
    return loader



### FRACAS data
----
**THIS WHOLE SECTION IS ENTIRELY NEW, WITH THE parseXML() FUNCTION ADAPTED FROM UNDER THE LINK IN THE COMMENT**

In [6]:
import csv
import xml.etree.ElementTree as ET

In [7]:
def parseXML(xmlfile):
    # adapted from https://www.geeksforgeeks.org/xml-parsing-python/
    tree = ET.parse(xmlfile)
    root = tree.getroot()
    problems = []
    counter = 0
  
    for item in root.findall('./problem'):
        counter += 1
        
        items = {}

        for child in item:
            if child.tag not in items:
                items[child.tag] = child.text.encode('utf8')
            else:  # if it is in items, meaning it's a problem with multiple premises
                items['multi'] = 'yes'
            
            # I estimated that from the category 5. Adjectives and onwards, those are semantic examples
            if counter <= 196:
                items['category'] = 'syntactic'
            else:
                items['category'] = 'semantic'
        
        if 'multi' not in items:  # if it is a 1-premise problem
            problems.append(items)
        else:
            continue
      
    return problems

In [8]:
def decode_problem(problem):
    problem['p'] = problem['p'].decode("utf-8").strip()
    problem['h'] = problem['h'].decode("utf-8").strip()
    del problem['q']

    return problem

In [9]:
def select_1_premise_problems(xmlfile):
    
    problems = parseXML(xmlfile)
    
    # removing unnecessary "features"
    for problem in problems:
        if 'note' in problem:
            del problem['note']
        if 'why' in problem:
            del problem['why']

    # picking the ones that are not undef, changing the relation name
    selected_problems = []
    for problem in problems:
        if problem['a'] == b' Yes ':
            problem['a'] = 'entailment'
            problem = decode_problem(problem)
            selected_problems.append(problem)
        elif problem['a'] == b' No ':
            problem['a'] = 'contradiction'
            problem = decode_problem(problem)
            selected_problems.append(problem)
        elif problem['a'] == b" Don't know ":
            problem['a'] = 'neutral'
            problem = decode_problem(problem)
            selected_problems.append(problem)
        else:
            continue
            
    return selected_problems

In [10]:
problems = select_1_premise_problems('Natlog Problems.xml')

In [11]:
full_fracas_frame = pd.DataFrame(problems)
# I need to rename the columns so that it works with our datasets and loaders from the section above.
full_fracas_frame.columns = ['premise', 'category', 'hypothesis', 'relation']

In [12]:
full_fracas_frame

,premise,category,hypothesis,relation
0,An Italian became the world's greatest tenor.,syntactic,There was an Italian who became the world's gr...,entailment
1,The really ambitious tenors are Italian.,syntactic,There are really ambitious tenors who are Ital...,entailment
2,No really great tenors are modest.,syntactic,There are really great tenors who are modest.,contradiction
3,Some great tenors are Swedish.,syntactic,There are great tenors who are Swedish.,entailment
4,Many great tenors are German.,syntactic,There are great tenors who are German.,entailment
...,...,...,...,...
160,It is true that ITEL won the contract in 1992.,semantic,ITEL won the contract in 1992.,entailment
161,It is false that ITEL won the contract in 1992.,semantic,ITEL won the contract in 1992.,contradiction
162,Smith saw Jones sign the contract.,semantic,Jones signed the contract.,entailment
163,Smith saw Jones sign the contract and his secr...,semantic,Smith saw Jones sign the contract.,entailment


----
Usually data is divided into train, test, dev in a 8:1:1 ratio. Since I will not have a train set here (I train on the SNLI
data and only finetune on FRACAS dev), that leaves us with test and dev at 1:1.

since I want it shuffled, but I want to keep the results reproducible for re-running the notebook, I will keep a fixed seed. 

----

In [13]:
random.Random(25).shuffle(problems)
print(problems)

[{'p': 'A few committee members are from Scandinavia.', 'category': 'syntactic', 'h': 'At least a few female committee members are from Scandinavia.', 'a': 'neutral'}, {'p': 'Smith wrote a report for two hours.', 'category': 'semantic', 'h': 'Smith wrote a report.', 'a': 'neutral'}, {'p': 'Smith wrote a novel in 1991.', 'category': 'semantic', 'h': 'Smith wrote it in 1992.', 'a': 'contradiction'}, {'p': 'Smith and Jones left the meeting.', 'category': 'semantic', 'h': 'Smith left the meeting.', 'a': 'entailment'}, {'p': 'No delegate finished the report on time.', 'category': 'syntactic', 'h': 'Some Scandinavian delegate finished the report on time.', 'a': 'contradiction'}, {'p': 'Some Irish delegates finished the survey on time.', 'category': 'syntactic', 'h': 'Some delegates finished the survey on time.', 'a': 'entailment'}, {'p': 'Several delegates got the results published in major national newspapers.', 'category': 'syntactic', 'h': 'Several delegates got the results published.', '

In [14]:
split_point = math.ceil(len(problems) / 2)

test_fracas = problems[:split_point]
dev_fracas = problems[split_point:]

In [15]:
test_fracas_frame = pd.DataFrame(test_fracas)
test_fracas_frame.columns = ['premise', 'category', 'hypothesis', 'relation']
dev_fracas_frame = pd.DataFrame(dev_fracas)
dev_fracas_frame.columns = ['premise', 'category', 'hypothesis', 'relation']

In [16]:
test_fracas_frame

,premise,category,hypothesis,relation
0,A few committee members are from Scandinavia.,syntactic,At least a few female committee members are fr...,neutral
1,Smith wrote a report for two hours.,semantic,Smith wrote a report.,neutral
2,Smith wrote a novel in 1991.,semantic,Smith wrote it in 1992.,contradiction
3,Smith and Jones left the meeting.,semantic,Smith left the meeting.,entailment
4,No delegate finished the report on time.,syntactic,Some Scandinavian delegate finished the report...,contradiction
...,...,...,...,...
78,"When Jones got his job at the CIA, he knew tha...",semantic,It is the case that Jones is not and will neve...,entailment
79,"Either Smith, Jones or Anderson signed the con...",syntactic,If Smith and Anderson did not sign the contrac...,entailment
80,In 1994 ITEL sent a progress report every month.,semantic,ITEL sent a progress report in July 1994.,entailment
81,Fido is not a small animal.,semantic,Fido is a large animal.,neutral


In [17]:
dev_fracas_frame

,premise,category,hypothesis,relation
0,Smith claimed he had costed his proposal and s...,syntactic,Jones claimed Smith had costed Jones' proposal.,neutral
1,Neither commissioner spends time at home.,syntactic,One of the commissioners spends a lot of time ...,contradiction
2,Several Portuguese delegates got the results p...,syntactic,Several delegates got the results published in...,entailment
3,Many delegates obtained interesting results fr...,syntactic,Many delegates obtained results from the survey.,entailment
4,At least three commissioners spend a lot of ti...,syntactic,At least three commissioners spend time at home.,entailment
...,...,...,...,...
77,Smith discovered a new species in 1991.,semantic,Smith discovered it in 1992.,contradiction
78,"John wanted to buy a car, and he did.",syntactic,John bought a car.,entailment
79,The people who were at the meeting all voted f...,syntactic,Everyone at the meeting voted for a new chairman.,entailment
80,Some great tenors are Swedish.,syntactic,There are great tenors who are Swedish.,entailment


In [18]:
# I will want to see the performance on syntactic vs. semantic examples, so I will divide the test set into those two
syn_test_fracas = []
sem_test_fracas = []

for item in test_fracas:
    if item["category"] == 'syntactic':
        syn_test_fracas.append(item)
    else:  # if category == semantic
        sem_test_fracas.append(item)

In [19]:
syn_test_fracas_frame = pd.DataFrame(syn_test_fracas)
syn_test_fracas_frame.columns = ['premise', 'category', 'hypothesis', 'relation']
sem_test_fracas_frame = pd.DataFrame(sem_test_fracas)
sem_test_fracas_frame.columns = ['premise', 'category', 'hypothesis', 'relation']

In [20]:
syn_test_fracas_frame

,premise,category,hypothesis,relation
0,A few committee members are from Scandinavia.,syntactic,At least a few female committee members are fr...,neutral
1,No delegate finished the report on time.,syntactic,Some Scandinavian delegate finished the report...,contradiction
2,Some Irish delegates finished the survey on time.,syntactic,Some delegates finished the survey on time.,entailment
3,Several delegates got the results published in...,syntactic,Several delegates got the results published.,entailment
4,Bill suggested to Frank's boss that they shoul...,syntactic,If it was suggested that Bill and Frank's boss...,neutral
5,Mary used her workstation.,syntactic,Mary is female.,entailment
6,An Italian became the world's greatest tenor.,syntactic,There was an Italian who became the world's gr...,entailment
7,John said Bill had hurt himself.,syntactic,Someone said John had been hurt.,neutral
8,Several great tenors are British.,syntactic,There are great tenors who are British.,entailment
9,Most Europeans who are resident in Europe can ...,syntactic,Most Europeans can travel freely within Europe.,neutral


In [21]:
sem_test_fracas_frame

,premise,category,hypothesis,relation
0,Smith wrote a report for two hours.,semantic,Smith wrote a report.,neutral
1,Smith wrote a novel in 1991.,semantic,Smith wrote it in 1992.,contradiction
2,Smith and Jones left the meeting.,semantic,Smith left the meeting.,entailment
3,Smith ran his own business for two years.,semantic,Smith ran his own business.,entailment
4,John has a genuine diamond.,semantic,John has a diamond.,entailment
5,Smith wrote a report in two hours.,semantic,Smith spent more than two hours writing the re...,contradiction
6,ITEL won more orders than APCOM lost.,semantic,APCOM lost some orders.,neutral
7,The PC-6082 is as fast as the ITEL-XZ.,semantic,The PC-6082 is fast.,neutral
8,Smith discovered new species for two years.,semantic,Smith discovered new species.,entailment
9,John is a former successful university student.,semantic,John is a university student.,neutral


# 2. Model

**This section is roughly the same as in Lab 5, with the exception of using PyTorch max pooling rather than a custom (and slower) function. I have also changed the embedding size relative to what was submitted in the draft submission of Lab 5.**

### Creating a representation of a sentence
----
I will not use my custom max pooling function, as it was rather slow. Instead, I will use the PyTorch one. 

In [22]:
num_words = 3
dimensions = 5
# A tensor for testing
test_tensor = torch.rand([batch_size, num_words, dimensions], dtype=torch.float64, device=device)
print(test_tensor)

tensor([[[0.9973, 0.2028, 0.5969, 0.3569, 0.7039],
         [0.8063, 0.2427, 0.9144, 0.1467, 0.3266],
         [0.0015, 0.0476, 0.3649, 0.4639, 0.4136]],

        [[0.7822, 0.2053, 0.1870, 0.6607, 0.3030],
         [0.5048, 0.9425, 0.7269, 0.9409, 0.1806],
         [0.0495, 0.0554, 0.9932, 0.1991, 0.2824]],

        [[0.9137, 0.6891, 0.0384, 0.7406, 0.8322],
         [0.4547, 0.2106, 0.3668, 0.8926, 0.6953],
         [0.6774, 0.6739, 0.8714, 0.4483, 0.8837]],

        [[0.8788, 0.1030, 0.8126, 0.4094, 0.7939],
         [0.5000, 0.0871, 0.8569, 0.1065, 0.6569],
         [0.1280, 0.5772, 0.6256, 0.0927, 0.0315]],

        [[0.1312, 0.5323, 0.5180, 0.7756, 0.2858],
         [0.8747, 0.2452, 0.7436, 0.6780, 0.2015],
         [0.6435, 0.3681, 0.9342, 0.1387, 0.5844]],

        [[0.6043, 0.6746, 0.0721, 0.6585, 0.7726],
         [0.1174, 0.1103, 0.5554, 0.8066, 0.0453],
         [0.8795, 0.6554, 0.6987, 0.9962, 0.5145]],

        [[0.2941, 0.8541, 0.0693, 0.0466, 0.4640],
         [0.3123, 0

In [23]:
torch.max(test_tensor, dim=1)[0]

tensor([[0.9973, 0.2427, 0.9144, 0.4639, 0.7039],
        [0.7822, 0.9425, 0.9932, 0.9409, 0.3030],
        [0.9137, 0.6891, 0.8714, 0.8926, 0.8837],
        [0.8788, 0.5772, 0.8569, 0.4094, 0.7939],
        [0.8747, 0.5323, 0.9342, 0.7756, 0.5844],
        [0.8795, 0.6746, 0.6987, 0.9962, 0.7726],
        [0.7993, 0.8541, 0.7961, 0.1573, 0.5613],
        [0.7206, 0.9582, 0.9299, 0.9839, 0.6607]], device='cuda:2',
       dtype=torch.float64)

### Combining sentence representations
----

Next, we need to combine the premise and hypothesis into one representation. We will do this by concatenating four tensors (the final size of our tensor $X$ should be ``(batch_size, 4d)`` where ``d`` is the number of dimensions that you use): 

$$X = [P; H; |P-H|; P \cdot H]$$

Here, what we do is concatenating P, H, P times H, and the absolute value of P minus H, then return the result.

In [24]:
# Test tensors (size of batch, num, dim)
t = torch.rand([2*batch_size, num_words, dimensions], dtype=torch.float64, device=device)
t1, t2 = torch.split(t, batch_size)
# Pooled test tensors (size of batch, dim)
pt1 = torch.max(t1, dim=1)[0]
pt2 = torch.max(t2, dim=1)[0]
print(pt1)
print(pt2)

tensor([[0.5785, 0.8087, 0.4113, 0.9270, 0.9243],
        [0.9588, 0.6624, 0.7774, 0.6892, 0.9450],
        [0.5717, 0.9277, 0.9457, 0.7634, 0.9692],
        [0.5089, 0.4516, 0.9349, 0.8483, 0.7984],
        [0.7900, 0.7996, 0.8440, 0.9910, 0.7734],
        [0.8106, 0.9239, 0.8820, 0.1656, 0.9829],
        [0.9365, 0.9001, 0.9855, 0.5686, 0.5854],
        [0.9030, 0.9901, 0.8790, 0.7217, 0.9446]], device='cuda:2',
       dtype=torch.float64)
tensor([[0.9047, 0.3731, 0.7511, 0.6076, 0.3318],
        [0.4196, 0.5730, 0.9987, 0.8928, 0.9750],
        [0.7057, 0.9369, 0.7609, 0.6481, 0.9697],
        [0.6345, 0.5356, 0.7137, 0.5761, 0.3219],
        [0.7932, 0.6299, 0.8212, 0.7591, 0.8341],
        [0.9976, 0.7579, 0.7937, 0.7028, 0.9906],
        [0.6720, 0.9173, 0.5918, 0.7554, 0.5298],
        [0.8108, 0.5774, 0.9457, 0.7739, 0.9728]], device='cuda:2',
       dtype=torch.float64)


In [25]:
def combine_premise_and_hypothesis(hypothesis, premise):
    
    batches = len(hypothesis)
    dims = len(hypothesis[0])
    final_dims = 4*dims

    new_tensors = []

    for i in range(0,batches):
        hyp = hypothesis[i]
        pre = premise[i]
    
        summed = torch.cat((pre,hyp))
        subtracted = pre - hyp
        multiplied = torch.mul(pre, hyp)
    
        new_tensors.append(torch.cat((summed, subtracted, multiplied)))
    
    output = torch.stack(new_tensors)
    
    return output

In [26]:
combine_premise_and_hypothesis(pt1, pt2)

tensor([[ 9.0475e-01,  3.7311e-01,  7.5106e-01,  6.0756e-01,  3.3175e-01,
          5.7850e-01,  8.0874e-01,  4.1133e-01,  9.2704e-01,  9.2431e-01,
          3.2625e-01, -4.3563e-01,  3.3973e-01, -3.1948e-01, -5.9256e-01,
          5.2340e-01,  3.0175e-01,  3.0893e-01,  5.6323e-01,  3.0664e-01],
        [ 4.1962e-01,  5.7305e-01,  9.9875e-01,  8.9281e-01,  9.7498e-01,
          9.5876e-01,  6.6244e-01,  7.7739e-01,  6.8921e-01,  9.4501e-01,
         -5.3914e-01, -8.9391e-02,  2.2136e-01,  2.0360e-01,  2.9966e-02,
          4.0231e-01,  3.7961e-01,  7.7642e-01,  6.1533e-01,  9.2136e-01],
        [ 7.0566e-01,  9.3693e-01,  7.6092e-01,  6.4810e-01,  9.6966e-01,
          5.7165e-01,  9.2769e-01,  9.4569e-01,  7.6336e-01,  9.6922e-01,
          1.3400e-01,  9.2360e-03, -1.8478e-01, -1.1527e-01,  4.4017e-04,
          4.0339e-01,  8.6918e-01,  7.1959e-01,  4.9474e-01,  9.3981e-01],
        [ 6.3452e-01,  5.3565e-01,  7.1366e-01,  5.7608e-01,  3.2185e-01,
          5.0894e-01,  4.5162e-01, 

### Creating the model
----

In [27]:
class SNLIModel(nn.Module):
    def __init__(self, word2idx, relation2idx, embedding_dim=64, hidden_size=128, padding_idx=1):
        super().__init__()
        self.vocab_size = len(word2idx)
        self.output_dim = len(relation2idx)
        self.hidden_size = hidden_size
        # your code goes here
        self.embeddings = nn.Embedding(self.vocab_size, embedding_dim, padding_idx=padding_idx) #
        self.LSTM = nn.LSTM(input_size=embedding_dim, hidden_size=self.hidden_size, num_layers=1, batch_first=True, bidirectional=True)
        self.classifier = nn.Linear(self.hidden_size*8, self.output_dim)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, premise, hypothesis):
        p = self.embeddings(premise)
        h = self.embeddings(hypothesis)
        
        lstm_p, (hidden, c) = self.LSTM(p)
        lstm_h, (hidden, c) = self.LSTM(h)
        
        p_pooled = torch.max(lstm_p, dim=1)[0]
        h_pooled = torch.max(lstm_h, dim=1)[0]
        
        ph_representation = combine_premise_and_hypothesis(h_pooled,p_pooled)
        ph_representation = self.dropout(ph_representation)  # is this at the right stage??
        
        predictions = self.classifier(ph_representation)
        
        return predictions

# 3. Training and testing

----
**This section re-uses the training loop from Lab 5 and adapts its evaluation loop into a function.**

In [28]:
train_dataset = InferenceDataset(train_data)
test_dataset = InferenceDataset(test_data)
# the dev set was used when testing if this works
dev_dataset = InferenceDataset(dev_data)

In [29]:
dev_fracas_dataset = InferenceDataset(dev_fracas_frame)
test_fracas_dataset = InferenceDataset(test_fracas_frame)
syn_test_fracas_dataset = InferenceDataset(syn_test_fracas_frame)
sem_test_fracas_dataset = InferenceDataset(sem_test_fracas_frame)

In [30]:
train_loader = dataloader(train_dataset, train_dataset.word2idx, train_dataset.pad_idx, train_dataset.unk_idx, batch_size=batch_size,shuffle=False)
test_loader = dataloader(test_dataset, train_dataset.word2idx, train_dataset.pad_idx, train_dataset.unk_idx, batch_size=batch_size,shuffle=False)
dev_loader = dataloader(dev_dataset, train_dataset.word2idx, train_dataset.pad_idx, train_dataset.unk_idx, batch_size=batch_size,shuffle=False)
dev_fracas_loader = dataloader(dev_fracas_dataset, train_dataset.word2idx, train_dataset.pad_idx, train_dataset.unk_idx, batch_size=batch_size,shuffle=False)
test_fracas_loader = dataloader(test_fracas_dataset, train_dataset.word2idx, train_dataset.pad_idx, train_dataset.unk_idx, batch_size=batch_size,shuffle=False)
syn_test_fracas_loader = dataloader(syn_test_fracas_dataset, train_dataset.word2idx, train_dataset.pad_idx, train_dataset.unk_idx, batch_size=batch_size,shuffle=False)
sem_test_fracas_loader = dataloader(sem_test_fracas_dataset, train_dataset.word2idx, train_dataset.pad_idx, train_dataset.unk_idx, batch_size=batch_size,shuffle=False)

### Training the model just on SNLI

In [31]:
model = SNLIModel(train_dataset.word2idx, train_dataset.labels).to(device)
loss_function = nn.CrossEntropyLoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    # train model
    total_loss = 0
    for i, batch in enumerate(train_loader):
        prems = batch[0]
        hyps = batch[1]
        rels = torch.Tensor(batch[2]).long().to(device)

        output = model(prems, hyps)
        
        loss = loss_function(output, rels)
        total_loss += loss.item()
        
        if i%2000==0:
            print(f' Batch {i} : Average Loss = {round(total_loss/(i+1),5)}')#, end='\r')
            
        # calculate gradients
        loss.backward()
        # update model weights
        optimizer.step()
        # reset gradients
        optimizer.zero_grad()

/home/gusszawma@GU.GU.SE/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


 Batch 0 : Average Loss = 1.355
 Batch 2000 : Average Loss = 0.99127
 Batch 4000 : Average Loss = 0.95516
 Batch 6000 : Average Loss = 0.92781
 Batch 8000 : Average Loss = 0.90663
 Batch 10000 : Average Loss = 0.89359
 Batch 12000 : Average Loss = 0.87893
 Batch 14000 : Average Loss = 0.86901
 Batch 16000 : Average Loss = 0.86039
 Batch 18000 : Average Loss = 0.85335
 Batch 20000 : Average Loss = 0.84687
 Batch 22000 : Average Loss = 0.84117
 Batch 24000 : Average Loss = 0.83558
 Batch 26000 : Average Loss = 0.83011
 Batch 28000 : Average Loss = 0.82555
 Batch 30000 : Average Loss = 0.82138
 Batch 32000 : Average Loss = 0.8175
 Batch 34000 : Average Loss = 0.8135
 Batch 36000 : Average Loss = 0.80956
 Batch 38000 : Average Loss = 0.80664
 Batch 40000 : Average Loss = 0.80359
 Batch 42000 : Average Loss = 0.80061
 Batch 44000 : Average Loss = 0.79729
 Batch 46000 : Average Loss = 0.79488
 Batch 48000 : Average Loss = 0.79227
 Batch 50000 : Average Loss = 0.78999
 Batch 52000 : Average L

In [32]:
torch.save(model, 'inference_VG.model')

In [33]:
import pickle
#so that when we load it back in, we can have access to the same word2idx etc.
with open("train_dataset.pickle","wb") as f:
    pickle.dump(train_dataset, f)

In [34]:
with open("train_dataset.pickle", 'rb') as f:
    train_dataset = pickle.load(f)
    
model = torch.load('inference_VG.model')
model.eval()

SNLIModel(
  (embeddings): Embedding(56258, 64, padding_idx=1)
  (LSTM): LSTM(64, 128, batch_first=True, bidirectional=True)
  (classifier): Linear(in_features=1024, out_features=4, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [35]:
def test_model(model, loader, name, divider):
    
    with torch.no_grad():

        correct = 0
        counter = 0
        for i, batch in enumerate(loader):
            test_output = model(batch[0], batch[1])
            # test_output = model(batch[1])
            test_output = torch.argmax(test_output, dim=1)
            targets = torch.tensor(batch[2], device=device)
            correct += torch.sum(test_output == targets)
            counter += len(test_output)

            test_accu = correct/counter
        
            if i%divider==0:
                print(f' Batch {i} : Average Test Accuracy on the {name} = {round(float(test_accu), 5)}')

    print(f'Total Test Accuracy on the {name} = {round(float(test_accu), 5)}')

In [36]:
test_model(model, test_loader, 'SNLI test set', 200)

 Batch 0 : Average Test Accuracy on the SNLI test set = 0.625
 Batch 200 : Average Test Accuracy on the SNLI test set = 0.72823
 Batch 400 : Average Test Accuracy on the SNLI test set = 0.73535
 Batch 600 : Average Test Accuracy on the SNLI test set = 0.72587
 Batch 800 : Average Test Accuracy on the SNLI test set = 0.72612
 Batch 1000 : Average Test Accuracy on the SNLI test set = 0.72765
 Batch 1200 : Average Test Accuracy on the SNLI test set = 0.72908
Total Test Accuracy on the SNLI test set = 0.7305


In [37]:
test_model(model, test_fracas_loader, 'FRACAS test set', 2)

 Batch 0 : Average Test Accuracy on the FRACAS test set = 0.375
 Batch 2 : Average Test Accuracy on the FRACAS test set = 0.5
 Batch 4 : Average Test Accuracy on the FRACAS test set = 0.525
 Batch 6 : Average Test Accuracy on the FRACAS test set = 0.53571
 Batch 8 : Average Test Accuracy on the FRACAS test set = 0.5
 Batch 10 : Average Test Accuracy on the FRACAS test set = 0.46988
Total Test Accuracy on the FRACAS test set = 0.46988


In [38]:
test_model(model, syn_test_fracas_loader, 'syntactic FRACAS test set', 2)

 Batch 0 : Average Test Accuracy on the syntactic FRACAS test set = 0.375
 Batch 2 : Average Test Accuracy on the syntactic FRACAS test set = 0.45833
 Batch 4 : Average Test Accuracy on the syntactic FRACAS test set = 0.45
Total Test Accuracy on the syntactic FRACAS test set = 0.45


In [39]:
test_model(model, sem_test_fracas_loader, 'semantic FRACAS test set', 2)

 Batch 0 : Average Test Accuracy on the semantic FRACAS test set = 0.75
 Batch 2 : Average Test Accuracy on the semantic FRACAS test set = 0.625
 Batch 4 : Average Test Accuracy on the semantic FRACAS test set = 0.475
Total Test Accuracy on the semantic FRACAS test set = 0.46512


### Fine-tuning the model on FRACAS

In [40]:
model.train()

for epoch in range(epochs):
    # train model
    total_loss = 0
    for i, batch in enumerate(dev_fracas_loader):
        prems = batch[0]
        hyps = batch[1]
        rels = torch.Tensor(batch[2]).long().to(device)

        output = model(prems, hyps)
        
        loss = loss_function(output, rels)
        total_loss += loss.item()
        
        if i%2==0:
            print(f' Batch {i} : Average Loss = {round(total_loss/(i+1),5)}')#, end='\r')
            
        # calculate gradients
        loss.backward()
        # update model weights
        optimizer.step()
        # reset gradients
        optimizer.zero_grad()

 Batch 0 : Average Loss = 1.22935
 Batch 2 : Average Loss = 1.08836
 Batch 4 : Average Loss = 1.03435
 Batch 6 : Average Loss = 1.06142
 Batch 8 : Average Loss = 1.04398
 Batch 10 : Average Loss = 1.03545
 Batch 0 : Average Loss = 1.26415
 Batch 2 : Average Loss = 1.03692
 Batch 4 : Average Loss = 0.94641
 Batch 6 : Average Loss = 0.9848
 Batch 8 : Average Loss = 0.97997
 Batch 10 : Average Loss = 0.98133
 Batch 0 : Average Loss = 1.5039
 Batch 2 : Average Loss = 1.17973
 Batch 4 : Average Loss = 1.04403
 Batch 6 : Average Loss = 1.04778
 Batch 8 : Average Loss = 1.03062
 Batch 10 : Average Loss = 1.02552


In [41]:
torch.save(model, 'inference_VG_finetuned.model')

In [42]:
finetuned_model = torch.load('inference_VG_finetuned.model')
model.eval()

SNLIModel(
  (embeddings): Embedding(56258, 64, padding_idx=1)
  (LSTM): LSTM(64, 128, batch_first=True, bidirectional=True)
  (classifier): Linear(in_features=1024, out_features=4, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [43]:
test_model(finetuned_model, test_loader, 'SNLI test set', 200)

 Batch 0 : Average Test Accuracy on the SNLI test set = 0.625
 Batch 200 : Average Test Accuracy on the SNLI test set = 0.7158
 Batch 400 : Average Test Accuracy on the SNLI test set = 0.7182
 Batch 600 : Average Test Accuracy on the SNLI test set = 0.71277
 Batch 800 : Average Test Accuracy on the SNLI test set = 0.71208
 Batch 1000 : Average Test Accuracy on the SNLI test set = 0.71429
 Batch 1200 : Average Test Accuracy on the SNLI test set = 0.71701
Total Test Accuracy on the SNLI test set = 0.719


In [44]:
test_model(finetuned_model, test_fracas_loader, 'FRACAS test set', 2)

 Batch 0 : Average Test Accuracy on the FRACAS test set = 0.375
 Batch 2 : Average Test Accuracy on the FRACAS test set = 0.54167
 Batch 4 : Average Test Accuracy on the FRACAS test set = 0.575
 Batch 6 : Average Test Accuracy on the FRACAS test set = 0.60714
 Batch 8 : Average Test Accuracy on the FRACAS test set = 0.55556
 Batch 10 : Average Test Accuracy on the FRACAS test set = 0.51807
Total Test Accuracy on the FRACAS test set = 0.51807


In [45]:
test_model(finetuned_model, syn_test_fracas_loader, 'syntactic FRACAS test set', 2)

 Batch 0 : Average Test Accuracy on the syntactic FRACAS test set = 0.25
 Batch 2 : Average Test Accuracy on the syntactic FRACAS test set = 0.375
 Batch 4 : Average Test Accuracy on the syntactic FRACAS test set = 0.4
Total Test Accuracy on the syntactic FRACAS test set = 0.4


In [46]:
test_model(finetuned_model, sem_test_fracas_loader, 'semantic FRACAS test set', 2)

 Batch 0 : Average Test Accuracy on the semantic FRACAS test set = 0.75
 Batch 2 : Average Test Accuracy on the semantic FRACAS test set = 0.625
 Batch 4 : Average Test Accuracy on the semantic FRACAS test set = 0.5
Total Test Accuracy on the semantic FRACAS test set = 0.51163


## Summary, evaluation, and discussion
#### Summary
The goal of this VG project was to verify and compare how well, in terms of accuracy, a BiLSTM model trained on the SNLI dataset performs on an SNLI test set and on the FRACAS test set (optionally split into semantic- and syntactic-based examples). The second step was to fine-tune the same model on a section of the FRACAS dataset, and perform the same evaluation, so that the results could be compared.  

The first step in this assignment was transforming the data from the FRACAS dataset into a format that would be similar to what I had in the SNLI dataset. This meant having to decide how to translate the relation labels as well as what to do with the examples that had more than one premise. All of that has been described in section 1 of the assignment.  
The second step was to create the model, for which I used the same architecture as in Assignment 5, with the exception of not using the custom max pooling function, as that one was really slow in comparison with the PyTorch one. I did make sure that the PyTorch one was giving me the output I wanted.  
The final step was training the model on SNLI and evaluating it, and then fine-tuning it on FRACAS and re-evaluating again. Both of the models have been saved (along with the train_dataset, whose word2idx is used to encode the words in loaders). The results of the evaluation can be seen below:  

| MODEL | SNLI | FRACAS (all) | FRACAS (syntactic) | FRACAS (semantic) |
|-----|-----|-------|-----------|---------|
| base | 0.7305 | 0.46988 | 0.45 | 0.46512 |
| finetuned | 0.719 | 0.51807 | 0.4 | 0.51163 |  

#### Evaluation 
It can be noticed that the base model (just trained on SNLI) performed better on the SNLI test set, but worse on all of the FRACAS ones. The fine-tuned model (first trained on SNLI, then trained more on FRACAS) performed worse on the SNLI test set, but better on the FRACAS ones. The difference in the peformance on SNLI is not major, and the one on FRACAS is bigger. However, these results are not always consistent when re-running the notebook, and generally oscillate around these values. It seems like fine-tuning on FRACAS has a rather marginal effect.  

#### Discussion
However, it is worth noting that the FRACAS dataset is significantly smaller than the SNLI train set. SNLI train has 550k examples, while our FRACAS dev - barely more than 80 (that is not 80k examples, that is 80 examples). This massive disproportion likely makes it so that the FRACAS examples do not contribute nearly as much as the SNLI ones, and perhaps balancing them out better would have been a better idea - this is difficult to do though, given that FRACAS itself is a small dataset. It is interesting to see though that it does have an effect, and on some runs of the notebook at least they do suggest that training a BiLSTM model on multiple kinds of inference datasets could improve its performance in some tasks without sacrificing too much of the performance elsewhere.  

Overall, given that we had 4 classes (the '-', 'contradiction', 'entailment', 'neutral' from SNLI), and given that '-' is a very rare one, the "stupid" baseline for this model would have been 33%. It performs better than that in all the categories, so I am fairly confident when I say it has learned something that lets it perform NLI - but whether it is something about the meaning or the structure of the data itself, I cannot say for certain.  

It seems that after fine-tuning, the model performs better on the semantic examples, and worse on the syntactic ones. While the first change wwas expected by me, the second one is rather surprising.

#### Possible future work
It would be really interesting to see the results with a more balanced SNLI-to-FRACAS ratio, seeing what the tradeoff is between gaining proficiency in FRACAS tasks and losing it in SNLI ones. It could also be interesting to test the model on other NLI datasets, or just the embeddings on other tasks. Overall I would also be interested in how NLI-based tasks could be used in training bigger, BERT-like models as one of the pre-training tasks, and what kind of meaning they would contribute - as clearly it also depends on the NLI dataset used.

#### Final remarks
When annotating the notebook for which parts I have done myself and which I have adapted, I felt like I have not done all that much, but I can promise that I did put quite some work into this little project, and hopefully it is sufficient. We have re-used similar elements in all of our neural network labs (e.g. the custom datasets and dataloaders which were initially done by Isac Boström from my group), and this project (and lab 5) is no exception in some of its sections. I found it particularly fun to have to think how to retrieve the FRACAS data and process it so that it would work with the model and the evaluation, and therein lies the most work I put into it. I also had to fix some minor issues with the model/training loop since relative to the draft submission of lab 5 there were some indexing errors when training on the training set. And, naturally, the discussion of the results is entirely mine, as is anything in the code that differs from lab 5 (unless stated otherwise). I hope that this is sufficient for the VG project, as discussed in e-mails before, and in case it is not, or you have any questions, issues, or comments, do not hesitate to contact me.

I am resubmitting this assignment due to a small mistake that was pointed out in the base version of assignment 5, where we forgot to set batch_first to True when setting our model up. This was changed and while it, surprisingly, did not influence the base model much, it did make a difference for whether the model performed better on semantic or syntactic FRACAS examples.

### Readings
[1] Samuel R. Bowman, Gabor Angeli, Christopher Potts, and Christopher D. Manning. 2015. A large annotated corpus for learning natural language inference. In Proceedings of the 2015 Conference on Empirical Methods in Natural Language Processing (EMNLP).

[2] Conneau, A., Kiela, D., Schwenk, H., Barrault, L., & Bordes, A. (2017). Supervised learning of universal sentence representations from natural language inference data. arXiv preprint arXiv:1705.02364.